In [ ]:
def distance(point1, point2):
    """Calculate the Euclidean distance between two D points."""
    return np.abs(np.sum(point1 - point2))

def ant_colony_optimization(points, n_ants, n_iterations, alpha, beta, evaporation_rate, Q):
    """Main function to run the ACO algorithm for TSP.
    points - a numpy array representing the coordinates of the cities in the TSP instance;
    alpha - A parameter controlling the influence of pheromone trails on ant decision-making;
    beta - a parameter controlling the influence of distance between cities on ant decision-making;
    evaporation_rate - the rate at which pheromone evaporates on each iteration. It's a value between 0 and 1;
    Q - A constant representing the quantity of pheromone deposited by each ant on its tour.
    """
    n_points = len(points)
    pheromone = np.ones((n_points, n_points))
    best_path = None
    best_path_length = np.inf

    for iteration in range(n_iterations):
        paths = []
        path_lengths = []

        for ant in range(n_ants): #Simulates the route of every single ant
            path, path_length = complete_tour(points, pheromone, alpha, beta, Q)
            paths.append(path)
            path_lengths.append(path_length)

            if path_length < best_path_length: #Updates the best path if a shorter one is found
                best_path = path
                best_path_length = path_length

        pheromone *= evaporation_rate
        update_pheromone(pheromone, paths, path_lengths, Q)

    plot_best_path(points, best_path) 
    print(f"Best path length: {best_path_length}") #Shows the best path length
    
def update_pheromone(pheromone, paths, path_lengths, Q):
    """Update the pheromone levels on the paths taken by the ants."""
    for path, path_length in zip(paths, path_lengths):
        for i in range(len(path) - 1):
            pheromone[path[i], path[i + 1]] += Q / path_length
        pheromone[path[-1], path[0]] += Q / path_length
        

def complete_tour(points, pheromone, alpha, beta, Q):
    """Simulates the entire tour for a single ant.
    Outputs the completed path array and its length"""
    n_points = len(points)
    visited = [False] * n_points
    current_point = np.random.randint(n_points)
    visited[current_point] = True
    path = [current_point]
    path_length = 0
    
    while False in visited:
        unvisited = np.where(np.logical_not(visited))[0]
        probabilities = calculate_probabilities(current_point, unvisited, pheromone, points, alpha, beta)
        next_point = np.random.choice(unvisited, p=probabilities)
        path.append(next_point)
        path_length += distance(points[current_point], points[next_point])
        visited[next_point] = True
        current_point = next_point

    return path, path_length

def plot_best_path(points, best_path):
    """Plot the best path found by the ants in 2D."""
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.scatter(points[:, 0], points[:, 1], color='blue')

    for i in range(len(best_path) - 1):
        ax.plot([points[best_path[i], 0], points[best_path[i + 1], 0]],
                [points[best_path[i], 1], points[best_path[i + 1], 1]],
                c='r', linestyle='-', linewidth=2, marker='o')

    ax.plot([points[best_path[-1], 0], points[best_path[0], 0]],
            [points[best_path[-1], 1], points[best_path[0], 1]],
            c='r', linestyle='-', linewidth=2, marker='o')

    ax.set_xlabel('X distance')
    ax.set_ylabel('Y distance')
    plt.title("Best Path Found by Ant Colony Optimization")
    plt.show()
